In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import os

data_path='D:\ArhamDQ-lab\WEB PROGRAMING\LOMBA\Satria Data/datasets'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)



{'men': 0, 'women': 1}
['men', 'women']
[0, 1]


In [2]:
img_size=32
data=[]
target=[]

facedata = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)           
                resized=cv2.resize(gray,(img_size,img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)


In [4]:
from numpy import asarray
from numpy import save
from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

np.save('data.npy',data)
np.save('new_target.npy',new_target)

In [5]:
import numpy as np

data=np.load('data.npy')
target=np.load('new_target.npy')

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint


noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=64



model=Sequential()
model.add((Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))
model.add((Conv2D(32, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [8]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
433/433 [==============================] - 26s 57ms/step - loss: 0.6796 - accuracy: 0.5800 - val_loss: 0.6766 - val_accuracy: 0.5739
Epoch 2/20
433/433 [==============================] - 25s 57ms/step - loss: 0.6687 - accuracy: 0.5922 - val_loss: 0.6630 - val_accuracy: 0.6005
Epoch 3/20
433/433 [==============================] - 26s 59ms/step - loss: 0.6560 - accuracy: 0.6091 - val_loss: 0.6466 - val_accuracy: 0.6176
Epoch 4/20
433/433 [==============================] - 25s 59ms/step - loss: 0.6501 - accuracy: 0.6126 - val_loss: 0.6398 - val_accuracy: 0.6237
Epoch 5/20
433/433 [==============================] - 24s 56ms/step - loss: 0.6440 - accuracy: 0.6180 - val_loss: 0.6500 - val_accuracy: 0.6179
Epoch 6/20
433/433 [==============================] - 25s 59ms/step - loss: 0.6392 - accuracy: 0.6149 - val_loss: 0.6350 - val_accuracy: 0.6216
Epoch 7/20
433/433 [==============================] - 26s 60ms/step - loss: 0.6345 - accuracy: 0.6210 - val_loss: 0.6303 - val_accuracy:

In [9]:
from keras.models import load_model
import cv2
import numpy as np

In [10]:
model = load_model('./training/model-001.model')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)

labels_dict={0:'Male',1:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(cap.isOpened()):

    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.03, 5)

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(32,32))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,32,32,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
    
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('Result',img)
    k=cv2.waitKey(1)
    
    if k==ord("q"):
        break
        
cv2.destroyAllWindows()

In [ ]:
cap.release()